# ```NEAT``` Tutorial #

### What is ```NEAT```? ###
NeuroEvolution of Augmenting Topologies (```NEAT```) is an evolutionary algorithm that creates artificial neural networks (```NN```).

### Instalation ###
```NEAT``` can be installed via ```pip3 install neat-python``` and ```pip3 install visualize```.

### High-Level Methodolgy ###
**Overview**<br>
In ```NEAT```, a population of individual ```genomes``` is maintained, and each ```genome``` contains 2 types of ```genes``` that describe how to build neural networks:
- ```Node```: specify a single ```neuron``` (all existing are mapped);
- ```Connection```: specify a ```connection``` between ```neurons``` from a previous layer to a following layer (all existing/potential are mapped);

Each ```node gene``` has the following attributes:
- ```ID```: unique identifier;
- ```Layer Type```: either ```sensor``` (input layer) or ```hidden``` (hidden layer);
- ```IO```: either ```input``` (```predictor```) or ```output``` (```response```);

Each ```connection gene``` has the following attributes:
- ```Input Node```: which node starts the connection;
- ```Output Node```: which node ends the connection;
- ```Weight```: the connection strength;
- ```Enabled```: either ```enabled``` (if both ```input node``` and ```output node``` exist; i.e. gene is manifested);
- ```Innovation```: identify the original historical ancestry;

A ```fitness function``` computes a single real number *indicating the quality of a genome* (where the higher the score, the higher ability to solve the problem). ```NEAT``` progresses through a pre-defined number of ```generations```, where each is produced by ```reproduction``` and ```mutation``` of the *most fit individuals of the previous generation*.

```Reproduction```/```Mutation``` *may add nodes/connections and/or change ```weights```*, making ```genomes``` (and their associated ```NN```s) more and more complex as ```generations``` progress. When the preset number of ```generations``` is reached, or at least one individual exceeds the the pre-defined ```fitness threshold```, the algorithm ends.

**Gene Tracking with ```Innovation```**<br>
To perform ```crossover```, ```NEAT``` must be able to tell which ```genes``` match up between individuals in the population. Two genes that have the same ```connection innovation``` (historical origin) *represent the same structure* (possibly with different ```weights```).

```Innovation``` is a global unique number that represent a chronology of every ```gene``` in the system (and sticks with it even *when passed to offspring*). Whenever a new gene appears (through structural ```mutation```), ```innovation``` is incremented.

```Genes``` that *do not match* are either ```disjoint``` or ```excess```, depending on whether they *occur within/outside the range of the other parent’s ```innovation``` numbers*. We can assume ```NEAT``` treats them without distinction and that the intention was to further investigate those options (which didn't happen).

When ```crossing over```, the ```genes``` with the same ```innovation``` are lined up. *Genes that do not match are inherited from the more fit parent* (or from both, randomly, if parents are equally fit). This results in a cheap process that doesn't need expensive topological analysis.

**Protecting Innovation through Speciation**<br> 
```NEAT``` *speciates the population*, so that individuals *compete primarily within their own niches instead of with the population at large*. Speciation guarantees that *topological innovations are protected and have time to optimize their structure before they compete with other niches in the population*.

Since the algorithm divides the population into species (based on topological similarity), the *number of ```disjoint```/```excess``` genes between a pair of ```genomes``` becomes a measure of their compatibility*. We can measure the compatibility distance (```delta```) as a linear combination of the number of excess (```E```) and disjoint (```D```) genes. 

If a ```genome```’s distance to a randomly chosen member of the species is less than ```delta_t``` (threshold), it's placed into this species (*```genomes``` are placed into the first species where this condition is satisfied*). In ```explicit fitness sharing``` *individuals in the same species must share the fitness of their niche, making it difficult for any species to take over the entire population*. Species fitnesses are first adjusted by dividing by the number of individuals in the species and, thus, setting the new number of individuals in species (they'll grow/shrink depending on whether their average adjusted fitness is above/below the population average).

### Configuration File Description ###
The configuration file (```.ini```) is in several sections as described below:

- **```[NEAT]```Section**
    - ```fitness_criterion```: computes the termination criterion from the set of genomes fitnesses (allowed: ```min```, ```max```, ```mean```);
    - ```fitness_threshold```: threshold for ```fitness_criterion```;
    - ```no_fitness_termination```: if *True*, ```fitness_criterion``` and ```fitness_threshold``` are ignored for termination (it happens by a maximum number of generations passed);
    - ```pop_size```: number of individuals in each generation;
    - ```reset_on_extinction```: if *True*, species become extinct due to stagnation and a new random one is created;
- **```[DefaultStagnation]```Section**
    - ```species_fitness_func```: computes species fitness (defaults: ```mean```; allowed: ```min```, ```max```, ```mean```, ```median```);
    - ```max_stagnation```: considers stagnant species that have not shown improvement after this number of generations (default:```15```);
    - ```species_elitism```: number of species that will be protected from stagnation (prevents total extinctions; default: ```0```);
- **```[DefaultReproduction]```Section**
    - ```elitism```: number of most-fit individuals in each species that will be preserved as-is for next generation (default: ```0```);
    - ```survival_threshold```: fraction for each species allowed to reproduce (default: ```0.2```);
    - ```min_species_size```: minimum number of genomes per species after reproduction (default: ```2```);
- **```[DefaultSpeciesSet]```Section**
    - ```compatibility_threshold```: individuals whose genomic distance is less than this threshold are considered to be in the same species;
- **```[DefaultGenome]```Section**
    - ```activation_default```: activation function assigned to nodes;
    - ```activation_mutate_rate```: probability that mutation will replace the node's activation function with a random member of ```activation_options``` (below);
    - ```activation_options```: list of the activation functions that may be used by nodes (default: ```sigmoid```);
    - ```aggregation_default```:  aggregation function assigned to nodes;
    - ```aggregation_mutate_rate```: probability that mutation will replace the node's aggregation function with a random member of ```aggregation_options```;
    - ```aggregation_options```: list of the aggregation functions that may be used by nodes (default: ```sum```);
    - ```bias_init_mean```: initial mean of the bias distribution for new nodes;
    - ```bias_init_stdev```: initial standard deviation of the bias distribution for new nodes;
    - ```bias_init_type```: initial bias distribution (allowed: ```gaussian```, ```uniform```);
    - ```bias_max_value```: maximum allowed bias;
    - ```bias_min_value```: minimum allowed bias;
    - ```bias_mutate_power```: standard deviation of zero-centered normal distribution from which bias mutation is drawn;
    - ```bias_mutate_rate```: probability that mutation will change the bias of a node by adding a random value;
    - ```bias_replace_rate```: probability that mutation will change the bias of a node with a new random value;
    - ```compatibility_disjoint_coefficient```: coefficient for the disjoint/excess gene counts' contribution to genomic distance;
    - ```compatibility_weight_coefficient```: coefficient for each weight, bias or response multiplier difference's contribution to genomic distance;
    - ```conn_add_prob```: probability that mutation will add a connection between existing nodes;
    - ```conn_delete_prob```: probability that mutation will delete an existing connection;
    - ```enabled_default```: default enabled attribute of newly created connections;
    - ```enabled_mutate_rate```: probability that mutation will replace the enabled status of a connection;
    - ```enabled_rate_to_false_add```: adds to the ```enabled_mutate_rate``` if connection is currently enabled;
    - ```enabled_rate_to_true_add```: adds to the ```enabled_mutate_rate``` if connection is currently not enabled;
    - ```feed_forward```: if *True*, generated networks are not allowed to have recurrent connections;
    - ```initial_connection```: specifies the initial connectivity of newly-created genomes (allowed: ```unconnected```, ```fs_neat_nohidden```, ```fs_neat_hidden```, ```full_nodirect```, ```full_direct```, ```partial_nodirect```, ```partial_direct```);
    - ```node_add_prob```: probability that mutation will add a new node (replacing an existing connection);
    - ```node_delete_prob```: probability that mutation will delete an existing node (and all its connections);
    - ```num_hidden```: number of hidden nodes to add to each genome in initial population;
    - ```num_inputs```: number of input nodes;
    - ```num_outputs```: number of output nodes;
    - ```response_init_mean```: initial mean of the response multiplier distribution;
    - ```response_init_stdev```: initial standard deviation of the response multiplier distribution;
    - ```response_init_type```: initial response multiplier distribution (allowed: ```gaussian```, ```uniform```);
    - ```response_max_value```: maximum allowed response multiplier;
    - ```response_min_value```: minimum allowed response multiplier;
    - ```response_mutate_power```: standard deviation of zero-centered normal distribution from which response multiplier mutation is drawn;
    - ```response_mutate_rate```: probability that mutation will change response multiplier of a node by adding a random value;
    - ```response_replace_rate```: probability that mutation will change response multiplier of a node with a new random value;
    - ```single_structural_mutation```: if *True*, only 1 structural mutation (addition/removal of a node/connection) will be allowed per genome per generation (default: *False*);
    - ```structural_mutation_surer```: if *True*, an attempt to add a node to a genome lacking connections will result in adding a connection instead / an attempt to add a connection tries to add a connection that already exists, that connection will be enabled;
    - ```weight_init_mean```: initial mean of the weights distribution for new connections;
    - ```weight_init_stdev```: initial standard deviation of the weights distribution for new connections;
    - ```weight_init_type```: initial weights distribution (allowed: ```gaussian```, ```uniform```);
    - ```weight_max_value```: maximum allowed weight value;
    - ```weight_min_value```: minimum allowed weight value;
    - ```weight_mutate_power```: standard deviation of zero-centered normal distribution from which weight mutation is drawn;
    - ```weight_mutate_rate```: probability that mutation will change weight of a node by adding a random value;
    - ```weight_replace_rate```: probability that mutation will change weight of a node with a new random value;

### Documentation ###
- Initial ```NEAT``` Paper (Section II) - [Link](https://nn.cs.utexas.edu/downloads/papers/stanley.cec02.pdf);
- ```NEAT``` Git Repo - [Link](https://github.com/CodeReclaimers/neat-python/tree/master);
---

### Importing Dependencies ###

In [1]:
import yaml
import os.path

import neat
import graphviz
import visualize # visualize.py (since 'visualize' library is not working)

### Main ###
**Example**: Implements a 2-input XOR function. XOR follows the behavior below:
| Input 1 | Input 2 | Output |
|---------|---------|--------|
| 0       | 0       | 0      |
| 0       | 1       | 1      |
| 1       | 0       | 1      |
| 1       | 1       | 0      |

In [2]:
# Reading the YAML config
with open("./neat.yaml", "r") as _stream:
    yaml_config = yaml.safe_load(_stream)

In [3]:
# 2-input XOR inputs and expected outputs
xor_inputs = [(0.0, 0.0), (0.0, 1.0), (1.0, 0.0), (1.0, 1.0)]
xor_outputs = [(0.0,), (1.0,), (1.0,), (0.0,)]

In [4]:
def eval_genomes(genomes, n): # MUST take only 2 arguments: 1) population, as a list of '(genome id, genome)' tuples
                              #                             2) maximum number of generations to run
    for genome_id, genome in genomes:
        genome.fitness = 4.0
        net = neat.nn.FeedForwardNetwork.create(genome, n)
        for xi, xo in zip(xor_inputs, xor_outputs):
            output = net.activate(xi)
            genome.fitness -= (output[0] - xo[0]) ** 2

In [7]:
# Load configuration file (with no file extension)
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     "neat_config_feedforward")
# Create the population
pop = neat.Population(config) # 'Population': implements the core evolution algorithm that follows the steps below;
                              #            1) Evaluate fitness of all genomes;
                              #            2) Check if the termination criterion is satisfied; exit if it is;
                              #            3) Generate the next generation from current population;
                              #            4) Partition the new generation into species based on genetic similarity;
                              #            5) Loop (go back to 1);
# Add a stdout reporter to show progress in the terminal
pop.add_reporter(neat.StdOutReporter(show_species_detail=False)) # 'StdOutReporter': outputs information about the run
                                                                 # 'show_species_detail': whether or not to show details about each species  
# Run until a solution is found
winner = pop.run(eval_genomes, n=300) # 'run': runs NEAT’s algorithm for at most 'n' generations
                                      # 'eval_genomes': fitness function
                                      # Important: It's assumed that fitness function doesn't modify the list of genomes, the genomes themselves, or the config object
# Display the winning genome
print("\nBest genome:\n{!s}".format(winner))
# Show output of the most fit genome against training data
print("\nOutput:")
winner_net = neat.nn.FeedForwardNetwork.create(winner, config) # 'create': receives a genome and returns its phenotype
for xi, xo in zip(xor_inputs, xor_outputs):
    output = winner_net.activate(xi) # Feeding inputs into the network and getting 'output'
    print("\tinput {!r}, expected output {!r}, got {!r}".format(xi, xo, output))


 ****** Running generation 0 ****** 

Population's average fitness: 2.25323 stdev: 0.33611
Best fitness: 2.98490 - size: (1, 2) - species 1 - id 107
Average adjusted fitness: 0.614
Mean genetic distance 1.199, standard deviation 0.411
Population of 150 members in 1 species
Total extinctions: 0
Generation time: 0.020 sec

 ****** Running generation 1 ****** 

Population's average fitness: 2.35073 stdev: 0.29630
Best fitness: 2.98490 - size: (1, 2) - species 1 - id 107
Average adjusted fitness: 0.594
Mean genetic distance 1.296, standard deviation 0.480
Population of 150 members in 1 species
Total extinctions: 0
Generation time: 0.111 sec (0.066 average)

 ****** Running generation 2 ****** 

Population's average fitness: 2.32132 stdev: 0.32887
Best fitness: 3.31651 - size: (2, 3) - species 1 - id 338
Average adjusted fitness: 0.409


Mean genetic distance 1.372, standard deviation 0.513
Population of 150 members in 1 species
Total extinctions: 0
Generation time: 0.103 sec (0.078 average)

 ****** Running generation 3 ****** 

Population's average fitness: 2.33288 stdev: 0.33940
Best fitness: 3.31651 - size: (2, 3) - species 1 - id 338
Average adjusted fitness: 0.442
Mean genetic distance 1.496, standard deviation 0.490
Population of 150 members in 1 species
Total extinctions: 0
Generation time: 0.072 sec (0.076 average)

 ****** Running generation 4 ****** 

Population's average fitness: 2.29953 stdev: 0.32669
Best fitness: 3.31651 - size: (2, 3) - species 1 - id 338
Average adjusted fitness: 0.275
Mean genetic distance 1.648, standard deviation 0.520
Population of 150 members in 1 species
Total extinctions: 0
Generation time: 0.063 sec (0.074 average)

 ****** Running generation 5 ****** 

Population's average fitness: 2.37772 stdev: 0.40148
Best fitness: 3.47822 - size: (3, 7) - species 1 - id 770
Average adjuste

**Interpreting the Results**
- ```Inputs```: we see 2 input nodes (```-1``` and ```-2```);
- ```Ouput```: we see 1 output node (```0```);
- ```Nodes```: we see 4 nodes (```0```, ```39```, ```62```, ```1213```);
- ```Connections```: we see 12 connections;
    - ```-2 -> 0```: input to output;
    - ```-2 -> 39```: input to layer;
    - ```-2 -> 62```: input to layer;
    - ```-2 -> 1213```: input to layer;
    - ```-1 -> 0```: input to output;
    - ```-1 -> 39```: input to layer;
    - ```-1 -> 62```: input to layer;
    - ```39 -> 0```: layer to output;
    - ```39 -> 62```: layer to layer;
    - ```62 -> 0```: layer to output;
    - ```62 -> 1213```: layer to layer;
    - ```1213 -> 0```: layer to output;

---